In [1]:
# CELL 1: SETUP
import pandas as pd
import numpy as np
import os
import gc # Garbage Collector to free up memory

# Define paths
PROCESSED_DIR = r"C:\Users\findo\OneDrive\Desktop\faers-dashboard\data\processed"

# 1. Load the "Anchor" Table: Demographics
print("Loading Demographics data...")
df_demo = pd.read_parquet(os.path.join(PROCESSED_DIR, 'all_demo.parquet'))

# 2. Load the Drug Table (we need this to find our specific drugs)
print("Loading Drug data...")
df_drug = pd.read_parquet(os.path.join(PROCESSED_DIR, 'all_drug.parquet'))

print(f"--- LOAD COMPLETE ---")
print(f"Total Demographic Rows: {len(df_demo):,}")
print(f"Total Drug Rows:        {len(df_drug):,}")

Loading Demographics data...
Loading Drug data...
--- LOAD COMPLETE ---
Total Demographic Rows: 2,048,518
Total Drug Rows:        9,923,900


In [2]:
# CELL 2: DEDUPLICATION

print(f"Rows before dedup: {len(df_demo):,}")

# 1. Convert caseid and caseversion to numeric to ensure correct sorting
# (Sometimes they load as strings, which sort incorrectly like "10" < "2")
df_demo['caseid'] = pd.to_numeric(df_demo['caseid'], errors='coerce')
df_demo['caseversion'] = pd.to_numeric(df_demo['caseversion'], errors='coerce')

# 2. Sort by Case ID and Version (Ascending)
# So the highest version is at the bottom for each case
df_demo.sort_values(by=['caseid', 'caseversion'], ascending=[True, True], inplace=True)

# 3. Keep only the LAST occurrence of each caseid
df_demo_clean = df_demo.drop_duplicates(subset=['caseid'], keep='last').copy()

print(f"Rows after dedup:  {len(df_demo_clean):,}")
print(f"Removed {len(df_demo) - len(df_demo_clean):,} duplicate versions.")

# Free up memory by deleting the messy original
del df_demo
gc.collect()

Rows before dedup: 2,048,518
Rows after dedup:  1,826,300
Removed 222,218 duplicate versions.


0

In [3]:
# CELL 3: IDENTIFY TARGET DRUGS (GLP-1s)

# 1. Clean the drug names for searching (lowercase, strip spaces)
df_drug['drugname_clean'] = df_drug['drugname'].str.lower().str.strip()

# 2. Define the Target List (Brands + Generics)

target_pattern = 'ozempic|wegovy|rybelsus|semaglutide|mounjaro|zepbound|tirzepatide'

# 3. Filter the Drug Table
# Condition A: The drug name matches our pattern
# Condition B: The drug is the "Primary Suspect" (PS)
print("Searching for GLP-1 cases...")
mask_target = (
    df_drug['drugname_clean'].str.contains(target_pattern, na=False, regex=True) & 
    (df_drug['role_cod'] == 'PS')
)

df_drug_glp1 = df_drug[mask_target].copy()

# 4. Get the list of relevant Case IDs
# These are the IDs we will use to fetch Demographics and Reactions
relevant_ids = df_drug_glp1['primaryid'].unique()

print(f"--- FILTER RESULTS ---")
print(f"Found {len(df_drug_glp1):,} rows matching GLP-1s (Primary Suspect).")
print(f"Unique Cases (PrimaryIDs): {len(relevant_ids):,}")

# Clean up memory
del df_drug
gc.collect()

Searching for GLP-1 cases...
--- FILTER RESULTS ---
Found 89,571 rows matching GLP-1s (Primary Suspect).
Unique Cases (PrimaryIDs): 89,567


51

In [4]:
# CELL 4: LOADING & MERGING OTHER TABLES

print("--- PREPARING SUB-TABLES ---")

# 1. Filter DEMO to just our target 89k cases
# We pick specific columns to keep the file size small
cols_demo = ['primaryid', 'caseid', 'event_dt', 'age', 'age_cod', 'sex', 'wt', 'rept_dt', 'occr_country']
df_demo_target = df_demo_clean[df_demo_clean['primaryid'].isin(relevant_ids)][cols_demo].copy()
print(f"Target Demographics: {len(df_demo_target):,}")

# 2. Load and Filter OUTCOMES (OUTC)
# We handle this carefully to avoid duplicates
print("Loading Outcomes...")
df_outc = pd.read_parquet(os.path.join(PROCESSED_DIR, 'all_outc.parquet'))
df_outc_target = df_outc[df_outc['primaryid'].isin(relevant_ids)].copy()

# Feature Engineering: Create simple flags instead of keeping raw rows
# If a case has 'DE' in the outc_cod column, they died.
# If 'HO', they were hospitalized.
print("Flagging Serious Outcomes...")
outc_flags = df_outc_target.groupby('primaryid')['outc_cod'].apply(set).to_frame('outc_set')

# Create binary columns (0 or 1)
outc_flags['death'] = outc_flags['outc_set'].apply(lambda x: 1 if 'DE' in x else 0)
outc_flags['serious'] = outc_flags['outc_set'].apply(lambda x: 1 if any(c in x for c in ['DE', 'HO', 'LT', 'OT']) else 0)
outc_flags['hospital'] = outc_flags['outc_set'].apply(lambda x: 1 if 'HO' in x else 0)

# Drop the set column, we just need the flags
outc_flags = outc_flags[['death', 'serious', 'hospital']]

# 3. Load and Filter REACTIONS (REAC)
print("Loading Reactions...")
df_reac = pd.read_parquet(os.path.join(PROCESSED_DIR, 'all_reac.parquet'))
df_reac_target = df_reac[df_reac['primaryid'].isin(relevant_ids)][['primaryid', 'pt']].copy()
print(f"Target Reactions: {len(df_reac_target):,}")

print("\n--- EXECUTING MERGE ---")
# Merge 1: Demo + Drug Information (Indication/Name)
# We take the drug info from our filtered drug table
df_step1 = pd.merge(df_demo_target, df_drug_glp1[['primaryid', 'drugname_clean']], on='primaryid', how='left')

# Merge 2: Add Outcome Flags
df_step2 = pd.merge(df_step1, outc_flags, on='primaryid', how='left')
# Fill NaN outcomes with 0 (Non-serious)
df_step2[['death', 'serious', 'hospital']] = df_step2[['death', 'serious', 'hospital']].fillna(0).astype(int)

# Merge 3: Add Reactions (This expands the rows!)
df_final = pd.merge(df_step2, df_reac_target, on='primaryid', how='inner')

print(f"--- FINAL DATASET ---")
print(f"Total Rows: {len(df_final):,}")
print("Columns:", df_final.columns.tolist())

# Clean up
del df_outc, df_reac, df_demo_clean
gc.collect()

--- PREPARING SUB-TABLES ---
Target Demographics: 87,462
Loading Outcomes...
Flagging Serious Outcomes...
Loading Reactions...
Target Reactions: 215,660

--- EXECUTING MERGE ---
--- FINAL DATASET ---
Total Rows: 207,725
Columns: ['primaryid', 'caseid', 'event_dt', 'age', 'age_cod', 'sex', 'wt', 'rept_dt', 'occr_country', 'drugname_clean', 'death', 'serious', 'hospital', 'pt']


0

In [5]:
# CELL 5: CLEANING & EXPORT

print("--- FEATURE ENGINEERING ---")

# 1. Normalize Age to Years
# FAERS uses codes: YR=Year, MON=Month, WK=Week, DY=Day
def normalize_age(row):
    age = row['age']
    unit = row['age_cod']
    
    # Return NaN if empty
    if pd.isna(age) or pd.isna(unit):
        return None
    
    try:
        age = float(age)
    except:
        return None
        
    if unit == 'YR':
        return age
    elif unit == 'MON':
        return age / 12
    elif unit == 'WK':
        return age / 52
    elif unit == 'DY':
        return age / 365
    elif unit == 'DEC': # Decade
        return age * 10
    else:
        return None

df_final['age_years'] = df_final.apply(normalize_age, axis=1)

# Filter out impossible ages (e.g., typos like 999 years)
df_final.loc[df_final['age_years'] > 110, 'age_years'] = None

# Create Age Groups (Recruiters love bins)
bins = [0, 18, 65, 110]
labels = ['Pediatric (<18)', 'Adult (18-64)', 'Elderly (65+)']
df_final['age_group'] = pd.cut(df_final['age_years'], bins=bins, labels=labels)

# 2. Infer Indication (Diabetes vs. Obesity) based on Brand Name
# This creates the "Business Insight" column
def infer_indication(drug_name):
    if pd.isna(drug_name): return 'Unknown'
    
    name = drug_name.lower()
    # Weight Loss Brands
    if 'wegovy' in name or 'zepbound' in name:
        return 'Obesity / Weight Loss'
    # Diabetes Brands
    elif 'ozempic' in name or 'mounjaro' in name or 'rybelsus' in name:
        return 'Diabetes'
    # Generic or Unspecified
    else:
        return 'Unspecified/Other'

df_final['indication_group'] = df_final['drugname_clean'].apply(infer_indication)

print("--- EXPORTING ---")
# 3. Save to CSV
# This is the file you will connect to Tableau/Power BI
output_path = os.path.join(PROCESSED_DIR, 'faers_glp1_final.csv')
df_final.to_csv(output_path, index=False)

print(f"SUCCESS! Dashboard data saved to: {output_path}")
print("Preview of final columns:")
print(df_final[['primaryid', 'sex', 'age_group', 'indication_group', 'pt', 'serious']].head())

--- FEATURE ENGINEERING ---
--- EXPORTING ---
SUCCESS! Dashboard data saved to: C:\Users\findo\OneDrive\Desktop\faers-dashboard\data\processed\faers_glp1_final.csv
Preview of final columns:
   primaryid sex      age_group indication_group  \
0  157323573   F  Adult (18-64)         Diabetes   
1  157323573   F  Adult (18-64)         Diabetes   
2  157323573   F  Adult (18-64)         Diabetes   
3  157323573   F  Adult (18-64)         Diabetes   
4  157323573   F  Adult (18-64)         Diabetes   

                                pt  serious  
0  Altered visual depth perception        0  
1             Heart rate increased        0  
2          Blood glucose increased        0  
3                 Feeling abnormal        0  
4                   Vision blurred        0  
